<a href="https://colab.research.google.com/github/seonghunYang/big_contest2020/blob/master/EDA10_%EC%9D%B4%EB%8F%99%ED%8F%89%EA%B7%A0%EB%B2%95%EC%9D%84_%ED%86%B5%ED%95%9C_%ED%94%BC%EC%B3%90_%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
import pandas as pd
path = './gdrive/My Drive/빅콘테스트2020/데이터/2020빅콘테스트 데이터분석분야-퓨쳐스리그_스포츠투아이_제공데이터(.CSV)_시즌별, 시트별 구분/'

In [70]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
team_hitter_2016 = pd.read_csv(path + '2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2016.csv', encoding='utf-8-sig')
team_hitter_2017 = pd.read_csv(path + '2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2017.csv', encoding='utf-8-sig')
team_hitter_2018 = pd.read_csv(path + '2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2018.csv', encoding='utf-8-sig')
team_hitter_2019 = pd.read_csv(path + '2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2019.csv', encoding='utf-8-sig')
team_hitter_2020 = pd.read_csv(path + '2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2020.csv', encoding='utf-8-sig')
team_hitter_list = [team_hitter_2016, team_hitter_2017, team_hitter_2018, team_hitter_2019, team_hitter_2020]
team_hitter = pd.concat(team_hitter_list)
team_hitter_list.append(team_hitter)

In [5]:
new_columns = "게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,타자,타수,타점,득점,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,실책,잔루,득점권타율,득점권타수,득점권안타"
new_columns = new_columns.split(',')
for team_h in team_hitter_list:
  team_h.columns = new_columns

In [6]:
for hitter_i in team_hitter_list:
  hitter_i['일자'] = pd.to_datetime(hitter_i['일자'],format='%Y%m%d', errors='raise')
  hitter_i['시즌'] = hitter_i['일자'].dt.year 
  
for hitter_i in team_hitter_list:
  hitter_i['단타'] = hitter_i['안타'] - (hitter_i['2루타'] + hitter_i['3루타'] + hitter_i['홈런'])

In [14]:
c_dic = {2020: [0.730, 0.759,	0.938, 1.284,	1.595,	1.959],
         2019: [0.721,	0.752,	0.945,	1.317,	1.653,	2.074],
         2018: [0.726,	0.753,	0.920,	1.242,	1.533,	1.857],
         2017: [0.728,	0.756,	0.927,	1.259,	1.558,	1.901],
         2016: [0.747,	0.774,	0.944,	1.273,	1.569,	1.898]}

c_dic2 = {2020: [0.345,	1.153, 4328, 56881],
         2019: [0.335,	1.242, 6548, 55612],
         2018: [0.351,	1.075, 7994, 56903],
         2017: [0.351,	1.107, 7682, 56881],
         2016: [0.362,	1.097, 8074, 57614]}

In [16]:
def runBaseP(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드']) & (team_hitter['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['출루율'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['안타'] + previous_sum['4구']  + previous_sum['사구']
    down = previous_sum['사구'] + previous_sum['타수'] + previous_sum['4구'] + previous_sum['희비']
    if down == 0:
        row['출루율'] = 0
        return row
    row['출루율'] = up / down
    return row

def longHitP(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드']) & (team_hitter['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['장타력'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['단타'] * 1 + previous_sum['2루타'] * 2 + previous_sum['3루타'] * 3 + previous_sum['홈런'] * 4
    down = previous_sum['타수']
    if down == 0:
        row['장타력'] = 0
        return row
    row['장타력'] = up / down

    return row
def hitP(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드']) & (team_hitter['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['타율'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['안타']
    down = previous_sum['타수']
    if down == 0:
        row['타율'] = 0
        return row
    row['타율'] = up / down
    return row

def createBABIP(row):
  previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드']) & (team_hitter['일자'] < row['일자'])].tail()
  if len(previous) == 0:
      row['BABIP'] = 0
      return row
  previous_sum = previous.sum()
  up = previous_sum['안타'] - previous_sum['홈런']
  down = previous_sum['타수'] - previous_sum['홈런'] - previous_sum['삼진'] + previous_sum['희비']
  if down == 0:
      row['BABIP'] = 0
      return row
  row['BABIP'] = up / down
  return row

def createBBK(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드']) & (team_hitter['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['BB/K'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['4구']
    down = previous_sum['삼진']
    if down == 0:
        row['BB/K'] = 0
        return row
    row['BB/K'] = up / down
    return row

def createK_BB(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드']) & (team_hitter['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['K%'] = 0
        row['BB%'] = 0
        return row
    previous_sum = previous.sum()
    up_k = previous_sum['삼진']
    up_BB = previous_sum['4구'] - previous_sum['고4']
    down = previous_sum['타자'] - previous_sum['고4']
    if down == 0:
        row['K%'] = 0
        row['BB%'] = 0
        return row
    row['K%'] = up_k / down
    row['BB%'] = up_BB / down
    return row

def createwOBA(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드'])& (team_hitter['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['wOBA'] = 0
        return row
    previous_sum = previous.sum()
    ct = c_dic[row['시즌']]
    up = (ct[0] * (previous_sum['4구'] - previous_sum['고4']) + ct[1] * previous_sum['사구'] + ct[2] * previous_sum['단타'] + ct[3] * previous_sum['2루타'] + ct[4] * previous_sum['3루타'] + ct[5] * previous_sum['홈런'] )
    down = (previous_sum['타수'] + previous_sum['4구'] - previous_sum['고4'] + previous_sum['사구'] + previous_sum['희비'])
    if down == 0:
        row['wOBA'] = 0
        return row
    row['wOBA'] = up / down
    return row

def createwRAA(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드'])& (team_hitter['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['wRAA'] = 0
        return row
    previous_sum = previous.sum()
    ct = c_dic2[row['시즌']]
    up = row['wOBA'] - ct[0]
    down = ct[1]
    row['wRAA'] = up / down * previous_sum['타자']
    return row

def createwRC(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드'])& (team_hitter['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['wRC'] = 0
        return row
    previous_sum = previous.sum()
    ct = c_dic2[row['시즌']]
    up = ct[2]
    down = ct[3]
    row['wRC'] = (up / down) * previous_sum['타자'] + row['wRAA']
    return row

def createHR(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드'])& (team_hitter['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['HR%'] = 0
        return row
    previous_sum = previous.sum()
    up_hr = previous_sum['홈런']
    down = previous_sum['타자'] - previous_sum['고4']
    if down == 0:
        row['HR%'] = 0
        return row
        
    row['HR%'] = up_hr / down
    return row

def createRC(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드']) & (team_hitter['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['RC'] = 0
        return row
    previous_sum = previous.sum()
    upA = previous_sum['안타']+previous_sum['4구']-previous_sum['도루실패']+previous_sum['사구']-previous_sum['병살타']
    upB = (previous_sum['단타']*1.125)+(1.69*previous_sum['2루타'])+(3.02*previous_sum['3루타'])+(3.73*previous_sum['홈런'])+ 0.29*(previous_sum['4구']-previous_sum['고4']+previous_sum['사구'])
    down = previous_sum['타수']+previous_sum['4구']+previous_sum['사구']+previous_sum['희비']+previous_sum['희타']
    if down == 0:
        row['RC'] = 0
        return row
    row['RC'] = ((2.4*down+upA)*(3*down+upB)/(9*down)) - (0.9*down)
    
    return row

def createRC27(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드']) & (team_hitter['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['RC27'] = 0
        return row
    previous_sum = previous.sum()
    down = previous_sum['타수'] - previous_sum['안타']+previous_sum['병살타']+previous_sum['도루실패']+previous_sum['희타']+previous_sum['희비']
    if down == 0:
        row['RC27'] = 0
        return row
    row['RC27'] = (row['RC']/down)*27
    
    return row


def createSPD(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드']) & (team_hitter['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['SPD'] = 0
        return row
    previous_sum = previous.sum()
    try:
        f1 = ((previous_sum['도루']+3)/(previous_sum['도루']+previous_sum['도루실패']+7)-0.4) * 20
        f2 = ((previous_sum['도루']+previous_sum['도루실패'])/(previous_sum['단타']+previous_sum['4구']+previous_sum['사구']))**0.5/0.07
        f3 = previous_sum['3루타']/(previous_sum['타수']-previous_sum['홈런']-previous_sum['삼진'])/ 0.02*10
        f4 = ((previous_sum['득점']-previous_sum['홈런'])/(previous_sum['안타']+previous_sum['4구']-previous_sum['홈런']-previous_sum['사구'])-0.1)/0.04
        f5 = (0.055-previous_sum['병살타']/(previous_sum['타수']-previous_sum['홈런']-previous_sum['삼진']))/0.005
    except ZeroDivisionError:
        row['SPD'] = 0
        return row
    row['SPD'] = (f1+f2+f3+f4+f5) / 5
    return row

def createXR(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드']) & (team_hitter['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['XR'] = 0
        return row
    previous_sum = previous.sum()
    try:
        Y1 = 0.5*previous_sum['단타']+0.72*previous_sum['2루타']+1.04*previous_sum['3루타']+1.44*previous_sum['홈런']
        Y2 = (previous_sum['사구']+previous_sum['4구']-previous_sum['고4'])*0.34+0.25*previous_sum['고4']+0.18*previous_sum['도루']-0.32*previous_sum['도루실패']
        Y3 = -(previous_sum['타수']-previous_sum['안타']-previous_sum['삼진'])*0.09-0.098*previous_sum['삼진']-0.37*previous_sum['병살타']+0.37*previous_sum['희비']+0.04*previous_sum['희타']
    except ZeroDivisionError:
        row['XR'] = 0
        return row
    row['XR'] = Y1+Y2+Y3
    return row

In [17]:
try:
  team_hitter = team_hitter.apply(runBaseP,axis=1)
  team_hitter = team_hitter.apply(longHitP,axis=1)
  team_hitter['ops'] = team_hitter['출루율'] + team_hitter['장타력']
  team_hitter = team_hitter.apply(hitP,axis=1)
  team_hitter['ISO'] = team_hitter['장타력'] - team_hitter['타율']
except:
  team_hitter.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA10-팀타자_이동평균법_세이버 매트릭스_합본.csv', encoding='utf-8-sig', index=False)
finally:
  team_hitter.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA10-팀타자_이동평균법_세이버 매트릭스_합본.csv', encoding='utf-8-sig', index=False)

In [18]:
try:
  team_hitter = team_hitter.apply(createBABIP,axis=1)
  team_hitter = team_hitter.apply(createBBK,axis=1)
  team_hitter = team_hitter.apply(createK_BB,axis=1)
  team_hitter = team_hitter.apply(createwOBA, axis=1)
except:
  team_hitter.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA10-팀타자_이동평균법_세이버 매트릭스_합본.csv', encoding='utf-8-sig', index=False)
finally:
  team_hitter.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA10-팀타자_이동평균법_세이버 매트릭스_합본.csv', encoding='utf-8-sig', index=False)

In [19]:
try:
  team_hitter = team_hitter.apply(createwRAA, axis=1)
  team_hitter = team_hitter.apply(createwRC, axis=1)
  team_hitter = team_hitter.apply(createHR, axis=1)
  team_hitter['GPA'] = (1.8 * team_hitter['출루율'] + team_hitter['장타력']) / 4 
except:
  team_hitter.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA10-팀타자_이동평균법_세이버 매트릭스_합본.csv', encoding='utf-8-sig', index=False)
finally:
  team_hitter.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA10-팀타자_이동평균법_세이버 매트릭스_합본.csv', encoding='utf-8-sig', index=False)

In [24]:
try:
  team_hitter = team_hitter.apply(createRC, axis=1)
  team_hitter = team_hitter.apply(createRC27, axis=1)
  team_hitter = team_hitter.apply(createSPD, axis=1)
  team_hitter = team_hitter.apply(createXR, axis=1)
except:
  team_hitter.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA10-팀타자_이동평균법_세이버 매트릭스_합본.csv', encoding='utf-8-sig', index=False)
finally:
  team_hitter.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA10-팀타자_이동평균법_세이버 매트릭스_합본.csv', encoding='utf-8-sig', index=False)

In [63]:
team_hitter.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA10-팀타자_이동평균법_세이버 매트릭스_합본.csv', encoding='utf-8-sig', index=False)

In [43]:
def createR(row):
    previous = team_hitter[(team_hitter['시즌'] == row['시즌']) & (team_hitter['팀코드'] == row['팀코드'])& (team_hitter['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['최근득점'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['득점']
    down = 5
        
    row['최근득점'] = up / down
    return row

In [64]:
team_hitter = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA10-팀타자_이동평균법_세이버 매트릭스_합본.csv', encoding='utf-8-sig')

In [60]:
team_hitter = team_hitter.apply(createR, axis=1)

In [65]:
team_hitter

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,타자,타수,타점,득점,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,실책,잔루,득점권타율,득점권타수,득점권안타,시즌,단타,최근출루율,최근장타력,최근ops,최근타율,최근ISO,최근BABIP,최근BB/K,최근K%,최근BB%,최근wOBA,최근wRAA,최근wRC,최근HR%,최근GPA,최근RC,최근RC27,최근SPD,최근XR,최근득점
0,20160401HHLG0,2016-04-01,LG,HH,0,B,47,42,4,5,9,2,0,1,2,1,1,0,4,0,0,11,0,0,8,0.333333,12,4,2016,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0
1,20160401HHLG0,2016-04-01,HH,LG,0,T,52,46,4,4,13,2,0,0,0,0,3,0,3,0,0,10,1,2,12,0.200000,15,3,2016,11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0
2,20160401HTNC0,2016-04-01,NC,HT,0,B,36,30,5,5,9,2,0,2,0,0,1,0,5,0,0,9,1,1,7,0.142857,7,1,2016,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0
3,20160401HTNC0,2016-04-01,HT,NC,0,T,38,34,3,4,8,3,0,1,0,0,0,0,3,0,1,10,1,0,7,0.100000,10,1,2016,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0
4,20160401KTSK0,2016-04-01,SK,KT,0,B,36,36,4,4,10,5,0,1,0,0,0,0,0,0,0,7,1,1,5,0.375000,8,3,2016,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,20200719LTSS0,2020-07-19,LT,SS,0,T,31,29,2,2,6,1,0,1,0,0,1,0,1,0,0,7,3,0,2,0.000000,2,0,2020,4,0.377551,0.471910,0.849461,0.320225,0.151685,0.354167,0.551724,0.146465,0.080808,0.383393,6.593049,21.658607,0.030303,0.287875,29.847191,6.151711,3.730115,29.278,7.0
6396,20200719OBHT0,2020-07-19,HT,OB,0,B,38,32,4,4,8,0,1,0,0,0,0,0,6,0,0,5,1,0,7,0.571429,7,4,2020,7,0.351648,0.370370,0.722019,0.271605,0.098765,0.320312,0.548387,0.166667,0.086022,0.335436,-1.551068,12.677515,0.016129,0.250834,20.742724,4.341500,1.608126,20.232,3.2
6397,20200719OBHT0,2020-07-19,OB,HT,0,T,41,37,8,8,11,3,0,1,2,1,0,0,2,1,2,4,0,0,6,0.363636,11,4,2020,7,0.375661,0.437870,0.813531,0.325444,0.112426,0.344156,0.764706,0.089005,0.068063,0.369455,4.051093,18.584030,0.010471,0.278515,26.727481,5.682220,5.779361,27.994,5.2
6398,20200719WOSK0,2020-07-19,SK,WO,0,B,37,30,3,4,6,0,0,0,0,0,0,0,7,0,0,10,0,0,9,0.250000,8,2,2020,6,0.340102,0.375000,0.715102,0.278409,0.096591,0.318841,0.444444,0.182741,0.081218,0.329731,-2.608846,12.380623,0.025381,0.246796,24.189227,4.910595,5.286321,25.352,5.8


In [62]:
team_hitter.columns = ['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '타자', '타수', '타점', '득점',
       '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비', '4구', '고4', '사구',
       '삼진', '병살타', '실책', '잔루', '득점권타율', '득점권타수', '득점권안타', '시즌', '단타', '최근출루율',
       '최근장타력', '최근ops', '최근타율', '최근ISO', '최근BABIP', '최근BB/K', '최근K%', '최근BB%', '최근wOBA', '최근wRAA',
       '최근wRC', '최근HR%', '최근GPA', '최근RC', '최근RC27', '최근SPD', '최근XR', '최근득점']

In [58]:
team_hitter

,게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,타자,타수,타점,득점,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,실책,잔루,득점권타율,득점권타수,득점권안타,시즌,단타,출루율,장타력,ops,타율,ISO,BABIP,BB/K,K%,BB%,wOBA,wRAA,wRC,HR%,GPA,RC,RC27,SPD,XR
0,20160401HHLG0,2016-04-01,LG,HH,0,B,47,42,4,5,9,2,0,1,2,1,1,0,4,0,0,11,0,0,8,0.333333,12,4,2016,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000
1,20160401HHLG0,2016-04-01,HH,LG,0,T,52,46,4,4,13,2,0,0,0,0,3,0,3,0,0,10,1,2,12,0.200000,15,3,2016,11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000
2,20160401HTNC0,2016-04-01,NC,HT,0,B,36,30,5,5,9,2,0,2,0,0,1,0,5,0,0,9,1,1,7,0.142857,7,1,2016,5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000
3,20160401HTNC0,2016-04-01,HT,NC,0,T,38,34,3,4,8,3,0,1,0,0,0,0,3,0,1,10,1,0,7,0.100000,10,1,2016,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000
4,20160401KTSK0,2016-04-01,SK,KT,0,B,36,36,4,4,10,5,0,1,0,0,0,0,0,0,0,7,1,1,5,0.375000,8,3,2016,4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6395,20200719LTSS0,2020-07-19,LT,SS,0,T,31,29,2,2,6,1,0,1,0,0,1,0,1,0,0,7,3,0,2,0.000000,2,0,2020,4,0.377551,0.471910,0.849461,0.320225,0.151685,0.354167,0.551724,0.146465,0.080808,0.383393,6.593049,21.658607,0.030303,0.287875,29.847191,6.151711,3.730115,29.278
6396,20200719OBHT0,2020-07-19,HT,OB,0,B,38,32,4,4,8,0,1,0,0,0,0,0,6,0,0,5,1,0,7,0.571429,7,4,2020,7,0.351648,0.370370,0.722019,0.271605,0.098765,0.320312,0.548387,0.166667,0.086022,0.335436,-1.551068,12.677515,0.016129,0.250834,20.742724,4.341500,1.608126,20.232
6397,20200719OBHT0,2020-07-19,OB,HT,0,T,41,37,8,8,11,3,0,1,2,1,0,0,2,1,2,4,0,0,6,0.363636,11,4,2020,7,0.375661,0.437870,0.813531,0.325444,0.112426,0.344156,0.764706,0.089005,0.068063,0.369455,4.051093,18.584030,0.010471,0.278515,26.727481,5.682220,5.779361,27.994
6398,20200719WOSK0,2020-07-19,SK,WO,0,B,37,30,3,4,6,0,0,0,0,0,0,0,7,0,0,10,0,0,9,0.250000,8,2,2020,6,0.340102,0.375000,0.715102,0.278409,0.096591,0.318841,0.444444,0.182741,0.081218,0.329731,-2.608846,12.380623,0.025381,0.246796,24.189227,4.910595,5.286321,25.352


In [52]:
import plotly.graph_objects as go

data = team_hitter[(team_hitter['팀코드'] == 'LG') & (team_hitter['시즌'] == 2018)].copy()
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=data['일자'], y=data['최근ops'], mode='lines', name='최근ops'
    )
)
fig.add_trace(
    go.Scatter(
        x=data['일자'], y=data['최근타율'], mode='lines', name='최근타율'
    )
)
fig.add_trace(
    go.Scatter(
        x=data['일자'], y=data['최근XR'], mode='lines', name='최근XR'
    )
)
fig.add_trace(
    go.Scatter(
        x=data['일자'], y=data['최근wOBA'], mode='lines', name='최근wOBA'
    )
)
fig.add_trace(
    go.Scatter(
        x=data['일자'], y=data['최근출루율'], mode='lines', name='최근출루율'
    )
)
fig.add_trace(
    go.Scatter(
        x=data['일자'], y=data['최근HR%'], mode='lines', name='최근HR'
    )
)
fig.add_trace(
    go.Scatter(
        x=data['일자'], y=data['최근RC27'], mode='lines', name='최근RC27'
    )
)
fig.add_trace(
    go.Scatter(
        x=data['일자'], y=data['최근득점'], mode='lines', name='최근득점'
    )
)
fig.update_layout(
  {
      "title": {
          "x": 0.5,
          "y": 0.9,
          "font": {
              "size": 15
          }
      },
      "showlegend": True,
      "xaxis": {
          "title": "일자"
        },
    }
)
fig.show()

In [53]:
import plotly.graph_objects as go

data = team_hitter[(team_hitter['팀코드'] == 'LG') & (team_hitter['시즌'] == 2018)].copy()
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=data['일자'], y=data['최근ops'], mode='lines', name='최근ops'
    )
)
fig.add_trace(
    go.Scatter(
        x=data['일자'], y=data['최근타율'], mode='lines', name='최근타율'
    )
)

fig.add_trace(
    go.Scatter(
        x=data['일자'], y=data['최근wOBA'], mode='lines', name='최근wOBA'
    )
)
fig.add_trace(
    go.Scatter(
        x=data['일자'], y=data['최근출루율'], mode='lines', name='최근출루율'
    )
)
fig.add_trace(
    go.Scatter(
        x=data['일자'], y=data['최근HR%'], mode='lines', name='최근HR'
    )
)
fig.add_trace(
    go.Scatter(
        x=data['일자'], y=data['최근득점'], mode='lines', name='최근득점'
    )
)
fig.update_layout(
  {
      "title": {
          "x": 0.5,
          "y": 0.9,
          "font": {
              "size": 15
          }
      },
      "showlegend": True,
      "xaxis": {
          "title": "일자"
        },
    }
)
fig.show()

In [68]:
import plotly.graph_objects as go

data = team_hitter[(team_hitter['팀코드'] == 'WO') & (team_hitter['시즌'] == 2018)].copy()
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=data['일자'], y=data['최근RC27'], mode='lines', name='최근RC27'
    )
)
fig.add_trace(
    go.Scatter(
        x=data['일자'], y=data['최근득점'], mode='lines', name='최근득점'
    )
)

fig.update_layout(
  {
      "title": {
          "x": 0.5,
          "y": 0.9,
          "font": {
              "size": 15
          }
      },
      "showlegend": True,
      "xaxis": {
          "title": "일자"
        },
    }
)
fig.show()

In [56]:
team_hitter = pd.read_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA10-팀타자_이동평균법_세이버 매트릭스_합본.csv', encoding='utf-8-sig')

 투수 구하기

In [71]:
def load_Data(name):
    years = ['2016','2017','2018','2019','2020']
    name_salt = '2020빅콘테스트_스포츠투아이_제공데이터_'
    PATH = path
    df = pd.DataFrame()
    for year in years:
        if year == '2016':
            df = pd.read_csv(PATH + name_salt + name+'_'+year+'.csv', encoding='CP949')
        else :
            new_df = pd.read_csv(PATH + name_salt + name+'_'+year+'.csv', encoding='CP949')
            df = pd.concat([df, new_df])
    return df

In [88]:
team_pitcher = load_Data('팀투수')

In [89]:
new_columns = "게임키,일자,팀코드,상대팀코드,더블헤더코드,초말,완투,결과,홀드,이닝*3,투구수,타자,타수,안타,2루타,3루타,홈런,도루,도루실패,희타,희비,4구,고4,사구,삼진,병살타,폭투,보크,실책,실점,자책점,득점권WHIP,2점차이하WHIP,345번타자WHIP"
new_columns = new_columns.split(',')

In [90]:
team_pitcher.columns = new_columns

In [91]:
team_pitcher['일자'] = pd.to_datetime(team_pitcher['일자'],format='%Y%m%d', errors='raise')
team_pitcher['시즌'] = team_pitcher['일자'].dt.year 

In [92]:
C_dic = {}
for year in range(2016,2021):
  team_pitcher_year = team_pitcher[team_pitcher['시즌'] == year].copy()
  team_pitcher_year = team_pitcher_year.sum() 
  C = (( 9 * team_pitcher_year['자책점'] - ( 13 * team_pitcher_year['홈런'] + 3 * (team_pitcher_year['4구'] + team_pitcher_year['사구'] - team_pitcher_year['고4']) - 2 * team_pitcher_year['삼진'])) / (team_pitcher_year['이닝*3'] / 3))
  C_dic[year] = C

In [111]:
def createERA(row):
    previous = team_pitcher[team_pitcher['팀코드'] == row['팀코드']].copy()
    previous = previous[(previous['시즌'] == row['시즌']) &(previous['일자'] < row['일자'])].tail()
    if len(previous) == 0:
      row['평균자책점'] = 0
      return row
    up = previous['자책점'].sum()
    down = previous['이닝*3'].sum() / 3
    if down == 0:
        row['평균자책점']
        return row
    row['평균자책점'] = (up * 9) / down
    return row

def createRA(row):
    previous = team_pitcher[(team_pitcher['팀코드'] == row['팀코드'])].copy()
    previous = previous[(previous['시즌'] == row['시즌']) &(previous['일자'] < row['일자'])].tail()
    if len(previous) == 0:
      row['RA/9'] = 0
      return row
    up = previous['실점'].sum()
    down = previous['이닝*3'].sum() / 3
    if down == 0:
        row['RA/9']
        return row
    row['RA/9'] = (up * 9) / down
    return row

def createERC(row):
    previous = team_pitcher[(team_pitcher['팀코드'] == row['팀코드'])].copy()
    previous = previous[(previous['시즌'] == row['시즌']) &(previous['일자'] < row['일자'])].tail()
    if len(previous) == 0:
      row['ERC'] = 0
      return row
    previous_sum = previous.sum()
    PTB = 0.89 * (1.255 * (previous_sum['안타'] - previous_sum['홈런']) + 4 * previous_sum['홈런']) + 0.475 * (previous_sum['4구'] + previous_sum['사구'] - previous_sum['고4'])
    ERC = 9* ( ((previous_sum['안타'] + previous_sum['4구'] + previous_sum['사구']) * PTB) / (previous_sum['타자'] * previous_sum['이닝*3'] / 3) ) - 0.56
    if ERC < 2.24:
      ERC = 9* ( ((previous_sum['안타'] + previous_sum['4구'] + previous_sum['사구']) * PTB) / (previous_sum['타자'] * previous_sum['이닝*3'] / 3) ) * 0.75
    row['ERC'] = ERC
    return row

def createFIP(row):
    previous = team_pitcher[(team_pitcher['시즌'] == row['시즌']) & (team_pitcher['팀코드'] == row['팀코드'])  & (team_pitcher['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['FIP'] = 0
        return row
    previous_sum = previous.sum()
    FIP = (( 13 * previous_sum['홈런'] + 3 * (previous_sum['4구'] + previous_sum['사구'] - previous_sum['고4']) - 2 * previous_sum['삼진']) / (previous_sum['이닝*3'] / 3)) + C_dic[row['시즌']]
    row['FIP'] = FIP
    return row

def createLOB(row):
    previous = team_pitcher[(team_pitcher['시즌'] == row['시즌']) & (team_pitcher['팀코드'] == row['팀코드'])  & (team_pitcher['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['LOB'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['안타'] + previous_sum['4구'] + previous_sum['사구'] - previous_sum['실점']
    down = previous_sum['안타'] + previous_sum['4구'] + previous_sum['사구'] - (1.4 * previous_sum['홈런'])
    if down == 0:
      row['LOB'] = 0
      return row 
    row['LOB'] = up / down
    return row

def createK_HR(row):
    previous = team_pitcher[(team_pitcher['시즌'] == row['시즌']) & (team_pitcher['팀코드'] == row['팀코드'])  & (team_pitcher['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['홈런율'] = 0
        row['삼진율'] = 0
        return row
    previous_sum = previous.sum()
    up_HR = previous_sum['홈런'] 
    up_k = previous_sum['삼진']
    down = previous_sum['타자'] - previous_sum['고4']
    if down == 0:
        row['홈런율'] = 0
        row['삼진율'] = 0
        return row
    row['홈런율'] = up_HR / down
    row['삼진율'] = up_k / down
    return row

def create9(row):
    previous = team_pitcher[(team_pitcher['시즌'] == row['시즌']) & (team_pitcher['팀코드'] == row['팀코드'])  & (team_pitcher['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['K9'] = 0
        row['BB9'] = 0
        row['HR9'] = 0
        row['KBB'] = 0
        return row
    previous_sum = previous.sum()
    up_K = previous_sum['삼진'] 
    up_BB = previous_sum['4구'] - previous_sum['고4']
    up_HR = previous_sum['홈런']
    down = previous_sum['이닝*3'] / 3
    if down == 0 or up_BB == 0 :
        row['K9'] = 0
        row['BB9'] = 0
        row['HR9'] = 0
        row['KBB'] = 0
        return row
    row['K9'] = up_K / down * 9
    row['BB9'] = up_BB / down * 9
    row['HR9'] = up_HR / down * 9
    row['KBB'] = up_K / up_BB
    return row

def createWhip(row):
    previous = team_pitcher[(team_pitcher['시즌'] == row['시즌']) & (team_pitcher['팀코드'] == row['팀코드'])  & (team_pitcher['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['whip'] = 0
        return row
    previous_sum = previous.sum()
    up = previous_sum['안타'] + previous_sum['4구']
    down = previous_sum['이닝*3'] / 3
    if down == 0:
      row['whip'] = 0
      return row 
    row['whip'] = up / down
    return row

def createPFR(row):
    previous = team_pitcher[(team_pitcher['시즌'] == row['시즌']) & (team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['PFR'] = 0
        return row
    previous_sum = previous.sum()
    try:
        pg = (previous_sum["삼진"]+previous_sum["4구"])/(previous_sum["이닝*3"]/3)
    except ZeroDivisionError:
        pg = 0
    row["PFR"] = pg
    return row
def createPG(row):
    previous = team_pitcher[(team_pitcher['시즌'] == row['시즌']) & (team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['P/G'] = 0
        return row
    previous_sum = previous.sum()
    try:
        pg = previous_sum["투구수"]/previous.count()[0]
    except ZeroDivisionError:
        pg = 0
    row["P/G"] = pg
    return row

def createPIP(row):
    previous = team_pitcher[(team_pitcher['시즌'] == row['시즌']) & (team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['P/IP'] = 0
        return row
    previous_sum = previous.sum()
    try:
        pg = previous_sum["투구수"]/(previous_sum["이닝*3"]/3)
    except ZeroDivisionError:
        pg = 0
    row["P/IP"] = pg
    return row
def createPPA(row):
    previous = team_pitcher[(team_pitcher['시즌'] == row['시즌']) & (team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['P/PA'] = 0
        return row
    previous_sum = previous.sum()
    try:
        pg = previous_sum["투구수"]/previous_sum["타자"]
    except ZeroDivisionError:
        pg = 0
    row["P/PA"] = pg
    return row

def createCYP(row):
    previous = team_pitcher[(team_pitcher['시즌'] == row['시즌']) & (team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['CYP'] = 0
        return row
    previous_sum = previous.sum()
    try:
        pg = (previous_sum["이닝*3"]/6-previous_sum["자책점"])+previous_sum["삼진"]/10+len(previous[previous['결과'] == 'W'])
    except ZeroDivisionError:
        pg = 0
    row["CYP"] = pg
    return row

def createFE(row):
    previous = team_pitcher[(team_pitcher['시즌'] == row['시즌']) & (team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['F-E'] = 0
        return row
    previous_mean = previous.mean()
    try:
        fe = previous_mean["FIP"]-previous_mean["평균자책점"]
    except ZeroDivisionError:
        fe = 0
    row["F-E"] = fe
    return row

def createRE(row):
    previous = team_pitcher[(team_pitcher['시즌'] == row['시즌']) & (team_pitcher['팀코드'] == row['팀코드']) & (team_pitcher['일자'] < row['일자'])].tail()
    if len(previous) == 0:
        row['R-E'] = 0
        return row
    previous_sum = previous.mean()
    try:
        fe = previous_sum["RA/9"]-previous_sum["평균자책점"]
    except ZeroDivisionError:
        fe = 0
    row["R-E"] = fe
    return row

In [94]:
try:
  team_pitcher = team_pitcher.apply(createERA, axis=1)
  team_pitcher = team_pitcher.apply(createRA, axis=1)
  team_pitcher = team_pitcher.apply(createERC, axis=1)
  team_pitcher = team_pitcher.apply(createFIP, axis=1)
except:
  team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA10-팀투수_이동평균법_세이버 매트릭스_합본.csv', encoding='utf-8-sig', index=False)
  print('오류발생')
finally:
  team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA10-팀투수_이동평균법_세이버 매트릭스_합본.csv', encoding='utf-8-sig', index=False)

In [99]:
team_pitcher1 = team_pitcher.copy()

In [97]:
try:
  team_pitcher = team_pitcher.apply(createLOB, axis=1)
  team_pitcher = team_pitcher.apply(createK_HR, axis=1)
  team_pitcher = team_pitcher.apply(create9, axis=1)
  team_pitcher = team_pitcher.apply(createWhip, axis=1)
except:
  team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA10-팀투수_이동평균법_세이버 매트릭스_합본.csv', encoding='utf-8-sig', index=False)
  print('오류발생')
finally:
  team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA10-팀투수_이동평균법_세이버 매트릭스_합본.csv', encoding='utf-8-sig', index=False)

In [109]:
len(team_pitcher[team_pitcher['결과'] == 'W'])

3167

In [101]:
try:
  team_pitcher = team_pitcher.apply(createPFR, axis=1)
  team_pitcher = team_pitcher.apply(createPG, axis=1)
  team_pitcher = team_pitcher.apply(createPIP, axis=1)
  team_pitcher = team_pitcher.apply(createPPA, axis=1)
except:
  team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA10-팀투수_이동평균법_세이버 매트릭스_합본.csv', encoding='utf-8-sig', index=False)
  print('오류발생')
finally:
  team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA10-팀투수_이동평균법_세이버 매트릭스_합본.csv', encoding='utf-8-sig', index=False)

오류발생


In [112]:
try:
  team_pitcher = team_pitcher.apply(createCYP, axis=1)
  team_pitcher = team_pitcher.apply(createFE, axis=1)
  team_pitcher = team_pitcher.apply(createRE, axis=1)
except:
  team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA10-팀투수_이동평균법_세이버 매트릭스_합본.csv', encoding='utf-8-sig', index=False)
  print('오류발생')
finally:
  team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA10-팀투수_이동평균법_세이버 매트릭스_합본.csv', encoding='utf-8-sig', index=False)

In [117]:
team_pitcher.to_csv('./gdrive/My Drive/빅콘테스트2020/EDA결과/EDA10-팀투수_이동평균법_세이버 매트릭스_합본.csv', encoding='utf-8-sig', index=False)

In [116]:
team_pitcher.columns = ['게임키', '일자', '팀코드', '상대팀코드', '더블헤더코드', '초말', '완투', '결과', '홀드', '이닝*3',
       '투구수', '타자', '타수', '안타', '2루타', '3루타', '홈런', '도루', '도루실패', '희타', '희비',
       '4구', '고4', '사구', '삼진', '병살타', '폭투', '보크', '실책', '실점', '자책점', '득점권WHIP',
       '2점차이하WHIP', '345번타자WHIP', '시즌', '최근평균자책점', '최근RA/9', '최근ERC', '최근FIP', '최근LOB',
       '최근홈런율', '최근삼진율', '최근K9', '최근BB9', '최근HR9', '최근KBB', '최근whip', '최근PFR', '최근P/G', '최근P/IP',
       '최근P/PA', '최근CYP', '최근F-E', '최근R-E']